In [1]:
from sklearn.svm import SVC

In [2]:
from sklearn.metrics import accuracy_score

In [3]:
import pandas as panda

In [4]:
import numpy as np

In [5]:
import matplotlib.pyplot as plt
import seaborn

In [14]:
import math

In [42]:
from tensorflow.keras.layers import Embedding, Dense, Input, Dropout, LSTM, Activation, Conv2D, Reshape, Average, Bidirectional

In [6]:
df = panda.read_csv("NAS.csv")

In [7]:
df = df.dropna()

In [8]:
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,2003-12-18,19.482599,19.596901,19.025499,19.139799,19.139799,4978496.0
1,2003-12-19,19.368299,19.425501,18.282801,18.454201,18.454201,1410901.0
2,2003-12-22,18.739901,18.739901,17.997101,18.054300,18.054300,137047.0
3,2003-12-23,17.997101,17.997101,17.368700,17.425800,17.425800,229418.0
7,2003-12-29,17.425800,17.425800,16.854500,17.254400,17.254400,196206.0
...,...,...,...,...,...,...,...
4248,2020-10-12,0.709000,0.710000,0.650000,0.676800,0.676800,48320475.0
4249,2020-10-13,0.676800,0.676800,0.600000,0.600600,0.600600,47786200.0
4250,2020-10-14,0.601000,0.640000,0.596200,0.626000,0.626000,37534949.0
4251,2020-10-15,0.626000,0.626000,0.585000,0.605000,0.605000,26737615.0


In [67]:
data = df.filter(['Close'])

In [69]:
dataset = data.values

array([[19.139799],
       [18.454201],
       [18.0543  ],
       ...,
       [ 0.626   ],
       [ 0.605   ],
       [ 0.6106  ]])

In [15]:
training_data_lenght = math.ceil(len(dataset) * .8)

In [20]:
scaler = MinMaxScaler(feature_range = (0,1))
scaled_data = scaler.fit_transform(dataset)

In [22]:
train_data = scaled_data[0:training_data_lenght , :]

x_train = []
y_train = []

for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])

In [23]:
x_train , y_train = np.array(x_train), np.array(y_train)

In [26]:
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_train.shape

(3315, 60, 1)

In [44]:
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape = (x_train.shape[1], 1)))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

In [45]:
model.compile(optimizer='adam', loss = 'mean_squared_error')

In [46]:
model.fit(x_train, y_train, batch_size = 1, epochs = 1)

3315/3315 [==============================] - 69s 21ms/step - loss: 0.0012


In [47]:
test_data = scaled_data[training_data_lenght - 60: , :]
x_test = []
y_test = dataset[training_data_lenght: , :]
for i in range(60, len(test_data)):
    x_test.append(test_data[i-60:i, 0])

In [48]:
x_test = np.array(x_test)

In [49]:
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [50]:
#Get the predicted values
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

In [51]:
#get the RMSE (feilmargin)
rmse = np.sqrt( np.mean( predictions - y_test )**2 )
rmse

0.380186636301846

In [55]:
#plot data
train = data[:training_data_lenght]
valid = data[training_data_lenght:]
valid['Predictions'] = predictions

valid

<ipython-input-55-c5cb2f1556d0>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid['Predictions'] = predictions


,Close,Predictions
3408,128.722000,132.692383
3409,125.922997,132.072571
3410,126.723000,130.293823
3411,125.009003,128.801834
3412,130.550995,127.296280
...,...,...
4248,0.676800,0.630923
4249,0.600600,0.558906
4250,0.626000,0.482392
4251,0.605000,0.426314


In [56]:
user_input = input("Pick a date")

Pick a date2020-10-5


In [63]:
df['prediction'] = valid.Predictions    

In [66]:
df.dropna()

,Date,Open,High,Low,Close,Adj Close,Volume,prediction
3408,2017-06-07,133.979004,134.093002,127.580002,128.722000,128.722000,814778.0,132.692383
3409,2017-06-08,129.694000,129.865005,125.694000,125.922997,125.922997,472537.0,132.072571
3410,2017-06-09,127.694000,127.694000,125.408997,126.723000,126.723000,249201.0,130.293823
3411,2017-06-12,126.836998,127.750999,124.723000,125.009003,125.009003,444550.0,128.801834
3412,2017-06-13,125.694000,130.722000,125.694000,130.550995,130.550995,636722.0,127.296280
...,...,...,...,...,...,...,...,...
4248,2020-10-12,0.709000,0.710000,0.650000,0.676800,0.676800,48320475.0,0.630923
4249,2020-10-13,0.676800,0.676800,0.600000,0.600600,0.600600,47786200.0,0.558906
4250,2020-10-14,0.601000,0.640000,0.596200,0.626000,0.626000,37534949.0,0.482392
4251,2020-10-15,0.626000,0.626000,0.585000,0.605000,0.605000,26737615.0,0.426314
